In [1]:
import csv
import numpy as np

signal = []
label = []

with open('C:\\Users\\yurui\\Desktop\\item\\heart\\data\\train.csv','r') as f:
    reader = csv.DictReader(f)
    for line in reader:
        signal.append([float(num) for num in line['heartbeat_signals'].split(',')])
        label.append(int(float(line['label'])))



In [7]:
import random
import numpy as np
import torch
from DataAdapter import *
import torch.utils.data as Data
import visdom
from filter import filter

batch_size = 128

#设置训练显卡
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)

signal_filt = []
train_split = 0.8
valid_split = 0.1

for sig in signal:
    signal_filt.append(filter(sig))

dataset = DataAdapter(signal_filt,label)
train_size = int(len(signal) * train_split) - int(len(signal) * valid_split)
valid_size = int(len(signal) * valid_split)
test_size = len(signal_filt) - train_size - valid_size
train_dataset,valid_dataset,test_dataset = Data.random_split(dataset,[train_size,valid_size,test_size])

train_loader = Data.DataLoader(train_dataset,batch_size = batch_size,shuffle = True,num_workers = 0)
valid_loader = Data.DataLoader(valid_dataset,batch_size = batch_size,shuffle = True,num_workers = 0)
test_loader = Data.DataLoader(test_dataset,batch_size = batch_size,shuffle = False,num_workers = 0)

print('Data Loading Finished')

#训练可视化
vis = visdom.Visdom(port = 8007) #python -m visdom.server -p 8007  建立visdom本地虚拟服务器的代码 cmd下运行
loss_curve = vis.line(
    X = np.array( [0] ),
    Y = np.array( [[0,0]] ),
    opts = dict(
           xlabel='epoch',
           ylabel='train_loss',
           legend=['train_loss','valid_loss'])
)
acc_curve = vis.line(
    X = np.array( [0] ),
    Y = np.array( [[0,0]] ),
    opts = dict(
           xlabel='epoch',
           ylabel='acc',
           legend=['train_acc','valid_acc'])
)

cuda:1
Setting up a new session...
Data Loading Finished


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class CNN(nn.Module):

    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Conv1d(in_channels = 1,out_channels = 32,kernel_size = 10,stride = 1,padding = 0)
        self.conv2 = nn.Conv1d(32,64,10,1)
        self.conv3 = nn.Conv1d(64,128,1,1)
        self.conv4 = nn.Conv1d(128,256,1,1)
        self.maxpool = nn.MaxPool1d(4)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.linear_unit = nn.Sequential(
            nn.Linear(2816,128),
            nn.ReLU(),
            nn.Linear(128,4),
            nn.Softmax(dim=1)
        )

    def forward(self,x):
        x = x.view(x.size(0),1,x.size(1))
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.maxpool(x)
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        x = self.maxpool(x)
        x = self.dropout(x)
        x = x.view(x.size(0),-1)
        x = self.linear_unit(x)
        return x

# x = torch.rand(1,205)
# model = CNN()
# y = model.forward(x)
# print(y.size())


In [9]:
import torch.nn as nn
import torch.optim as optim 
from train_model import *
from test_model import *
import time

model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

best_loss = 3
model.to(device)
        
for epoch in range(1,201):
    time_all=0
    start_time = time.time()
    train_loss,train_acc = train_model(train_loader, model, criterion, optimizer,device)
    time_all = time.time()-start_time
    valid_loss,valid_acc = test_model(valid_loader,criterion,model,device)
    print('- Epoch: %d - Train_loss: %.5f - Train_acc: %.5f - Val_loss: %.5f - Val_acc: %5f - T_Time: %.3f' %(epoch,train_loss,train_acc,valid_loss,valid_acc,time_all))

    try:                                         
        vis.line(
                X = [epoch],
                Y = [[train_loss,valid_loss]],
                win = loss_curve,
                update='append'
                )    
    except:
        print('visdom error......')

    try:                                       
        vis.line(
                X = [epoch],
                Y = [[train_acc,valid_acc]],
                win = acc_curve,
                update='append'
                )      
    except:
        print('visdom error......')

        
    if valid_loss < best_loss:
        best_loss = valid_loss
        print('Find better model in Epoch {0}, saving model.'.format(epoch))
        torch.save(model.state_dict(), './best_model.pt')

print('Training Finished')

 - Train_loss: 0.75108 - Train_acc: 0.99260 - Val_loss: 0.75609 - Val_acc: 0.987441 - T_Time: 8.483
- Epoch: 127 - Train_loss: 0.75128 - Train_acc: 0.99240 - Val_loss: 0.75470 - Val_acc: 0.989122 - T_Time: 8.499
- Epoch: 128 - Train_loss: 0.75131 - Train_acc: 0.99235 - Val_loss: 0.75474 - Val_acc: 0.988825 - T_Time: 8.508
- Epoch: 129 - Train_loss: 0.75101 - Train_acc: 0.99274 - Val_loss: 0.75477 - Val_acc: 0.988627 - T_Time: 8.586
- Epoch: 130 - Train_loss: 0.75108 - Train_acc: 0.99263 - Val_loss: 0.75557 - Val_acc: 0.987935 - T_Time: 8.483
- Epoch: 131 - Train_loss: 0.75128 - Train_acc: 0.99243 - Val_loss: 0.75597 - Val_acc: 0.987540 - T_Time: 8.427
- Epoch: 132 - Train_loss: 0.75124 - Train_acc: 0.99244 - Val_loss: 0.75569 - Val_acc: 0.987737 - T_Time: 8.462
- Epoch: 133 - Train_loss: 0.75089 - Train_acc: 0.99279 - Val_loss: 0.75471 - Val_acc: 0.988924 - T_Time: 8.438
- Epoch: 134 - Train_loss: 0.75077 - Train_acc: 0.99290 - Val_loss: 0.75436 - Val_acc: 0.989221 - T_Time: 8.457
Find

In [17]:
from test_model_for_ali import test_model_for_ali
import torch

model = CNN()
model.load_state_dict(torch.load('best_model.pt'))
model.eval()

test_sum,test_acc = test_model_for_ali(test_loader,model,device)
print(test_sum)
print(test_acc)

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 1; 11.00 GiB total capacity; 8.39 GiB already allocated; 3.17 MiB free; 8.51 GiB reserved in total by PyTorch)

In [10]:
import csv
import os
import pandas as pd
from filter import filter

ipath = 'C:\\Users\\yurui\\Desktop\\item\\heart\\data'
opath = 'C:\\Users\\yurui\\Desktop\\item\\heart\\output'
signal = []

with open(os.path.join(ipath,'testA.csv'),'r') as f:
    reader = csv.DictReader(f)
    for line in reader:
        signal.append(filter([float(num) for num in line['heartbeat_signals'].split(',')]))


result = od.read_csv(os.path.join(ipath,'sample_submit.csv'))
result['label_0'] = res[0]
result['label_1'] = res[1]
result['label_2'] = res[2]
result['label_3'] = res[3]
result.to_csv(os.path.join(opath,'sample_submit.csv'),index = False)

ModuleNotFoundError: No module named 'pandas'